In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import xlrd
import os

In [20]:
# I will provide data below. This is the data on order placement

In [28]:
data=pd.read_table('/Users/yiyezhang/Documents/Data/NYPData/HF/data/pull2/order_2012_2018.csv',sep=',')

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [33]:
data.iloc[0:5]

,name,clientvisitguid,clientguid,createdwhen,ordersetname,enterrole,typecode,userguid
1,PT/INR,9001281187200270,9000008107000200,2018-12-24 18:17:49.457,NaN,Resident,Diagnostic,9000166173701190
2,Phosphorus,9000680486800270,9000148027900200,2014-11-23 12:21:08.407,AM Lab Order Set,Resident,Diagnostic,9000029802301190
3,Rosuvastatin Tab,9000804658800270,9000194512500200,2015-10-29 21:41:33.060,NaN,Physician,Medication,9000062832701190
4,Activated Partial Thromboplastin Time,9001094171000270,9000250181100200,2017-12-09 07:58:44.370,Heparin Drip Order Set (Adult),PA,Diagnostic,9000048549501190
5,Discharge Patient,9000882412200270,9000032365500200,2016-05-27 11:38:58.363,NaN,NP,Other,3000001218119001


In [62]:
data_task=pd.read_table('/Users/yiyezhang/Documents/Data/NYPData/HF/data/order_2012_2018_task.csv',sep=',')

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [32]:
len(data)

3706762

In [34]:
data['clientguid'].nunique()

8778

In [31]:
data=data[data['enterrole'].isin(['PA', 'Resident', 'Nurse', 'NP', 'Physician', 'VerifyChemoTherapy',
       'Fellow', 'Medical Resident', 'Attending Physician',
       'Registered Nurse', 'Nurse Anesthetist'])]

In [30]:
data['enterrole'].unique()

array(['None', 'Resident', 'Physician', 'PA', 'NP', 'Nurse',
       'Medical Student', 'Pharmacist', 'Dietician',
       'Attending Physician', 'VerifyChemoTherapy', 'Medical Resident',
       'Fellow', 'PT/OT', 'Nutritionist', 'Dietetic Intern',
       'Registered Nurse', 'Chemo Pharmacist', 'Nurse Anesthetist',
       'Respiratory Therapist', 'NP Student'], dtype=object)

In [26]:
#clientvisitguid is the visit's ID
#clientguid is the patient's ID. one patient ID can have multiple visitIDs.
#userguid is the care provider ID. enterrole is the care provider's role
#createdwhen is the time of order creation
#name is the order's name
#order set name is the name of the order set that the order comes from
#typecode is the type of the order
#orderguid is the ID of the individual order

In [35]:
data['userguid']=data['userguid'].astype(int)
data['clientvisitguid']=data['clientvisitguid'].astype(int)

In [36]:
data['createdwhen'] = data['createdwhen'].astype('datetime64[h]')  #change to m/s for minutes and seconds

In [37]:
len(data['userguid'].unique())

7510

In [39]:
data['year'] = pd.DatetimeIndex(data['createdwhen']).year

In [41]:
data.groupby(['year'])[['clientvisitguid']].nunique()

,clientvisitguid
year,
2012,3
2013,241
2014,1890
2015,2406
2016,3031
2017,3211
2018,3313


In [32]:
#below, I am creating a summary of userguid by visit and order

In [42]:
visit_user=data.groupby(['userguid','clientvisitguid'])[['name']].count()

In [43]:
visit_user=visit_user.reset_index()

In [44]:
visit_user.columns.values

array(['userguid', 'clientvisitguid', 'name'], dtype=object)

In [45]:
len(visit_user)

343852

In [46]:
visit_user_grp_by_user_mean=visit_user.groupby(['userguid'])[['name']].sum()

In [47]:
mean_user_orders=visit_user_grp_by_user_mean.reset_index()

In [48]:
len(mean_user_orders['userguid'].unique())

7510

In [41]:
#remove outliers

In [49]:
mean_user_orders=mean_user_orders[(np.abs(stats.zscore(mean_user_orders['name']) < 3))]

In [50]:
mean_user_orders['name'].mean()

305.5550206327373

In [51]:
mean_user_orders['name'].std()

679.0618733641938

In [52]:
mean_user_orders['name'].min()

1

In [53]:
mean_user_orders.loc[mean_user_orders['name'].idxmax()]

userguid    9000166177501190
name                    4320
Name: 6438, dtype: int64

this is the 1st file export

In [54]:
mean_user_orders.to_csv(path+"order_2012_2018_ordercount.csv")

filter data by only including orders from order sets

In [55]:
data_OS=data[data['userguid'].isin(mean_user_orders['userguid'])]

In [56]:
data_OS=data[data['ordersetname'].notna()]

In [57]:
len(data_OS)

1675917

In [58]:
len(data_OS['clientvisitguid'].unique())

13697

In [59]:
len(data_OS['userguid'].unique())

3087

In [60]:
user_role=data[['userguid','enterrole']].drop_duplicates()

this is the 2nd file export

In [61]:
user_role.to_csv(path+"2012_2018_user_role.csv")

In [62]:
user_OS=data_OS.groupby(['userguid'])[['ordersetname']].count()

In [63]:
user_OS=user_OS.reset_index()

In [64]:
user_OS['ordersetname'].mean()

542.8950437317784

In [65]:
user_OS['ordersetname'].std()

893.4792686446523

this is the 3rd file export

user_OS.to_csv(path+"order_2012_2018_OScount.csv")

1. using 2012_2018_user_role.csv, order_2012_2018_OScount.csv and order_2012_2018_ordercount.csv you will create a table with userid/#orders/#order set orders/%order set orders. 
2. Then you will create identify MD providers who are in the 1st and last quarters in terms of %order set orders
3. file created in 2 will be used later 

In [66]:
len(data_OS)

1675917

In [67]:
data_OS['userguid'].nunique()

3087

In [20]:
path = os.path.expanduser('~/Documents/Data/NYPData/HF/')

this is the diagnosis data

In [138]:
diag = pd.read_csv(path + 'data/pull2/diag_2012_2018_pull2.csv',sep=',',names=['clientguid','clientvisitguid','createdwhen','typecode','shortname','icd10code','icd9code','userguid','year'])

/Users/yiyezhang/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [139]:
diag = diag.iloc[1:]

In [140]:
diag.head()

,clientguid,clientvisitguid,createdwhen,typecode,shortname,icd10code,icd9code,userguid,year
1,9000001012800200,9000745799000270,2015-05-25 22:25:38.277,Admitting Dx,BACK PAIN/FATIGUE,NaN,NaN,9000026324701190,NaN
2,9000001012800200,9000745799000270,2015-05-26 04:22:55.117,ED Diagnosis,Back pain,M54.9,724.5,0,NaN
3,9000001012800200,9000939526300270,2016-10-27 10:50:51.737,Acute Dx,Shortness of breath,R06.02,786.05,9000027857501190,NaN
4,9000001012800200,9000939526300270,2016-10-27 20:27:36.003,Chronic Dx,Nonischemic cardiomyopathy,I42.9,425.4,0,NaN
5,9000001012800200,9000939526300270,2016-10-27 20:27:57.997,Chronic Dx,"Chronic kidney disease, stage 3",N18.3,585.3,0,NaN


In [141]:
print(len(diag))

393427


In [142]:
diag['userguid']=diag['userguid'].astype(int)
diag['clientvisitguid']=diag['clientvisitguid'].astype(int)

In [145]:
diag['createdwhen'] = pd.to_datetime(diag['createdwhen'])

In [146]:
diag['createdwhen'] = diag['createdwhen'].astype('datetime64[h]')

In [147]:
diag['year'] = pd.DatetimeIndex(diag['createdwhen']).year
diag.groupby(['year'])[['clientvisitguid']].nunique()

,clientvisitguid
year,
2012,1
2013,243
2014,1882
2015,2409
2016,3048
2017,3219
2018,3324


demographic data

In [129]:
demographics = pd.read_csv(path + 'data/pull2/demographic_2012_2018_pull2.csv',sep=',',names=["clientguid",'clientvisitguid','ethnicity1','ethnicity2','YOB','language','gender','race','maritalstatus'])

In [131]:
demographics = demographics.iloc[1:]

In [152]:
demographics.head()

,clientguid,clientvisitguid,ethnicity1,ethnicity2,YOB,language,gender,race,maritalstatus
1,9000001014700200,9000721880800270,N,N,1947,English,Female,White,Single
2,9000001052300200,9000731006600270,N,NaN,1925,English,Female,Other Not Described,Widower
3,9000001053600200,9001199818100270,N,NaN,1961,English,Male,Black/African Am.,Single
4,9000001060100200,9001092703500270,U,NaN,1918,English,Male,Declined,Married
5,9000001086000200,9000974781900270,N,N,1934,English,Female,White,Single


In [133]:
demographics['clientguid']=demographics['clientguid'].astype(int)
demographics['clientvisitguid']=demographics['clientvisitguid'].astype(int)

In [134]:
len(data[data['clientvisitguid'].isin(demographics['clientvisitguid'])]['clientvisitguid'].unique())

13410

In [135]:
demographics['clientguid'].nunique()

8606

In [72]:
#diag['icd10top3']=diag['icd10code'].str.split('.').str[0]

characterise patient diagnoses

In [88]:
datadic={}
for row in diag.itertuples():
    datadic[row.ClientVisitGUID]={}
    datadic[row.ClientVisitGUID]['Initial']=[]
    datadic[row.ClientVisitGUID]['Last']=[]
    

In [91]:
for row in diag.itertuples():
    if row.typecode=='ED Diagnosis':
        datadic[row.ClientVisitGUID]['Initial'].append(row.ICD10Code)
    elif row.typecode=='Acute Dx':
        datadic[row.ClientVisitGUID]['Initial'].append(row.ICD10Code)
    elif row.typecode=='RegAdmittingDx':
        datadic[row.ClientVisitGUID]['Initial'].append(row.ICD10Code)
    elif row.typecode=='Working Dx':
        datadic[row.ClientVisitGUID]['Initial'].append(row.ICD10Code)
    elif row.typecode=='Admitting Dx':
        datadic[row.ClientVisitGUID]['Initial'].append(row.ICD10Code)
    elif row.typecode=='Principal Dx':
        datadic[row.ClientVisitGUID]['Last'].append(row.ICD10Code)
    elif row.typecode=='Billing Dx':
        datadic[row.ClientVisitGUID]['Last'].append(row.ICD10Code)
    elif row.typecode=='Discharge Dx':
        datadic[row.ClientVisitGUID]['Last'].append(row.ICD10Code)
    elif row.typecode=='RegPrincipal Dx':
        datadic[row.ClientVisitGUID]['Last'].append(row.ICD10Code)

these are list of ICD codes for specific projects (examples)
'R57.0', 'I09.81', 'I11.0', 'I13.0', 'I13.2', 'I50', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9', 'I50.22', 'I50.32', 'I50.42', 'I50.2', 'I50.20', 'I50.21', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.43', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9'

In [92]:
initlast=[]
for pid in datadic:
    for i in datadic[pid]['Initial']:
        for j in datadic[pid]['Last']:
            if i==j and i in ('R57.0', 'I09.81', 'I11.0', 'I13.0', 'I13.2', 'I50', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9', 'I50.22', 'I50.32', 'I50.42', 'I50.2', 'I50.20', 'I50.21', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.33', 'I50.4', 'I50.40', 'I50.41', 'I50.43', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.4', 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.9'):
                initlast.append(pid)

In [93]:
initlast=list(set(initlast)) 

In [94]:
len(initlast)

2345

In [149]:
diag_EDDC=diag[diag['clientvisitguid'].isin(initlast)]

In [150]:
len(diag_EDDC)

72090

In [97]:
order_EDDC=data[data['clientvisitguid'].isin(initlast)]

In [39]:
stats=order_EDDC[order_EDDC['typecode']=='Medication'].groupby(['name'])['name'].count()

In [40]:
stats.sort_values()

name
Posaconazole Inj +R+                                        1
Trimethobenzamide Inj                                       1
Benzocaine Spray 20% Oral                                   1
DivalproEX Delayed Release Oral                             1
Bendamustine Inj                                            1
Donnatal Oral Elixir                                        1
Barium Sulfate 2.1% Oral Susp                               1
PCRA Bupivacaine 0.125% in 0.9% sodium chloride             1
PCEA HYDROmorphone 10 mcg/mL in 0.9% sodium chloride        1
Bacitracin/Neomycin/Polymyxin B Opth Oint                   1
Vancomycin for Peritoneal Dialysis                          1
PCA Morphine Sulfate (CONTINUOUS)                           1
Varicella Virus Vaccine Inj +R+ (Varivax)                   1
EPINEPHrine IM (1:1,000)                                    1
Enalapril Oral Suspension Test Dose                         1
Oseltamivir Tab                                             1
Epi

In [153]:
demographics_EDDC=demographics[demographics['clientvisitguid'].isin(initlast)]

these are the 4th-6th files to export

In [99]:
order_EDDC.to_csv(path+"order_EDDC_2012_2018.csv", index=False)

In [100]:
order_EDDC['clientvisitguid'].nunique()

2345

In [151]:
diag_EDDC.to_csv(path+"diag_EDDC_2012_2018.csv", index=False)

In [154]:
demographics_EDDC.to_csv(path+"demographics_EDDC_2012_2018.csv", index=False)

In [127]:
len(demographics_EDDC)

2284

this is the patientvisit data

In [103]:
clientvisit=pd.read_csv(path + 'data/clientvisit_2012_2018_jupiter.csv',sep=',',names=["dischargedisposition","dischargedtm","admitdtm","clientvisitguid","clientguid","service"])

In [104]:
clientvisit=clientvisit.iloc[1:]

In [105]:
clientvisit['clientguid']=clientvisit['clientguid'].astype(int)
clientvisit['clientvisitguid']=clientvisit['clientvisitguid'].astype(int)

In [106]:
clientvisit['admitdtm'] = clientvisit['admitdtm'].astype('datetime64[h]')

In [107]:
clientvisit['dischargedtm'] = clientvisit['dischargedtm'].astype('datetime64[h]')

In [108]:
clientvisit=clientvisit[~clientvisit['dischargedtm'].isna()]

In [109]:
clientvisit_EDDC=clientvisit[clientvisit['clientvisitguid'].isin(initlast)]

this is the 7th file to export

In [110]:
clientvisit_EDDC.to_csv(path+"visit_EDDC_2012_2018.csv", index=False)

this is the 8th file to export

In [111]:
clientlocation=pd.read_csv(path + 'data/clientlocation_2012_2018.csv',sep=',')

In [112]:
clientlocation['clientvisitguid']=clientlocation['clientvisitguid'].astype(int)
clientlocation['createdwhen'] = clientlocation['createdwhen'].astype('datetime64[h]')

In [113]:
clientlocation=clientlocation.sort_values(['clientvisitguid', 'createdwhen'], ascending=[True, True])
clientlocation = clientlocation.drop_duplicates()

In [114]:
clientlocation[clientlocation['clientvisitguid']==9001285016600270]

,name,code,typecode,clientvisitguid,createdwhen
82960,A3,L |EMERGENCY |090 ...,Bed,9001285016600270,2018-12-31 00:00:00
90708,LMH ED QR,L |EMERGENCY |LQK |,Unit,9001285016600270,2018-12-31 00:00:00
82037,A1L,L |EMERGENCY |090 ...,Bed,9001285016600270,2018-12-31 01:00:00
64904,LMH-ERH-03,L |EMERGENCY |ERDH ...,Bed,9001285016600270,2018-12-31 04:00:00
68441,LMH4C-0417-02,L |INPATIENT |4C ...,Bed,9001285016600270,2018-12-31 08:00:00


In [115]:
clientlocation_EDDC=clientlocation[clientlocation['clientvisitguid'].isin(initlast)]

In [116]:
clientlocation_EDDC.head()

,name,code,typecode,clientvisitguid,createdwhen
8517,Adult ED,A |EMERGENCY |AEQKER |,Holding Unit,9000556943700270,2013-11-22 13:00:00
8321,ED AREA C,A |EMERGENCY |AEM018 |,Holding Unit,9000556943700270,2013-11-22 16:00:00
25646,G04N-4206-B,A |INPATIENT |G04N ...,Bed,9000556943700270,2013-11-22 18:00:00
22445,G04S-4410-A,A |INPATIENT |G04S ...,Bed,9000556943700270,2013-11-24 02:00:00
8323,G04N-4222-B,A |INPATIENT |G04N ...,Bed,9000556943700270,2013-11-26 21:00:00


In [117]:
clientlocation_EDDC.to_csv(path+"location_EDDC_2012_2018.csv", index=False)

In [ ]:
order_EDDC_OSonly=order_EDDC[order_EDDC['ordersetname'].notna()]

In [ ]:
order_EDDC_OSonly.shape

In [ ]:
user_order_EDDC_OSonly=order_EDDC_OSonly.groupby(['userguid'])[['ordersetname']].count()

In [ ]:
user_order_EDDC_OSonly=user_order_EDDC_OSonly.reset_index()

In [ ]:
user_order_EDDC=order_EDDC.groupby(['userguid'])[['name']].count()

In [ ]:
user_order_EDDC=user_order_EDDC.reset_index()

**these are 9th and 10th files to export
1. using 2012_2018_user_role.csv, user_order_EDDC_2012_2018.csv and user_order_EDDC_OSonly_2012_2018.csv you will create a table with userid/#orders/#order set orders/%order set orders. 
2. Then you will create identify MD providers who are in the 1st and last quarters in terms of %order set orders
3. file created in 2 will be used later 

In [ ]:
user_order_EDDC.to_csv(path+"user_order_EDDC_2012_2018.csv", index=False)

In [ ]:
user_order_EDDC_OSonly.to_csv(path+"user_order_EDDC_OSonly_2012_2018.csv", index=False)

In [ ]:
order_EDDC['typecode'].unique()

This is the file generated above**

In [ ]:
user_quartile=pd.read_excel('/Users/yiyezhang/Dropbox/Grant-and-IRB/AHRQ R03/Analysis/EDDC_order_2012_2018_analysis.xlsx', sheetname='MDOnly_1order_peryear')

In [ ]:
OS1Q=user_quartile[user_quartile['Q1']==1]

In [ ]:
OS3Q=user_quartile[user_quartile['Q4']==1]

In [ ]:
OS3Q.shape

In [ ]:
data_1Q=order_EDDC[order_EDDC['userguid'].isin(OS1Q['userguid'])]

In [ ]:
data_3Q=order_EDDC[order_EDDC['userguid'].isin(OS3Q['userguid'])]

In [ ]:
10 and 11th files to export - these can be read in Disco to create process maps of orders

In [ ]:
data_1Q.to_csv(path+"EDDC_18_89_order_acuteHF_2018_1Q.csv")

In [ ]:
data_3Q.to_csv(path+"EDDC_18_89_order_acuteHF_2018_3Q.csv")

In [ ]:
diag_3Q=diag[diag['userguid'].isin(OS3Q['userguid'])]

In [ ]:
diag_1Q=diag[diag['userguid'].isin(OS1Q['userguid'])]

In [ ]:
diag_1Q['icd10top3']=diag_1Q['icd10code'].str.split('.').str[0]

In [ ]:
diag_3Q['icd10top3']=diag_3Q['icd10code'].str.split('.').str[0]

In [ ]:
diag_1Q.to_csv(path+"EDDC_diag_acuteHF_2018_1Q.csv")

In [ ]:
diag_3Q.to_csv(path+"EDDC_diag_acuteHF_2018_3Q.csv")

In [ ]:
order_EDDC['name'].nunique()

extra: creating a list of patients who have only 1 diagnosis

In [ ]:
HFonly=[]
for pid in datadic:
    if pid in initlast and len(datadic[pid]['Last'])==1:
        HFonly.append(pid)

In [ ]:
len(HFonly)

In [ ]:
userguid_EDDC_18_89=[0, 9000028999901190, 9000025620701190, 9000011771101190, 9000022087901190, 9000017223901190, 3000001301119001, 9000118474801190, 9000028209201190, 9000002916401190, 9000174052401190, 9000017559601190, 9000216010801190, 9000025831301190, 9000053888901190, 9000060493701190, 9000206720901190, 9000043267801190, 9000193795801190, 9000119607001190, 9000134122201190, 9000041450201190, 9000212023001190, 9000119018201190, 9000118483501190, 9000166176301190, 9000015341101190, 9000001952301190, 9000190496301190, 9000215328301190, 9000022332801190, 9000172502401190, 9000211158401190, 9000119359701190, 9000116646101190, 9000036031701190, 9000049241301190, 9000185996501190, 4000001216119001, 9000202690601190, 9000211164201190, 9000181289001190, 9000065167401190, 9000137820201190, 9000089487401190, 9000182876201190, 9000131167101190, 9000183007101190, 9000191403901190, 9000047096701190, 9000018527101190, 9000201823101190, 9000119598801190, 9000166165201190, 9000211554001190, 9000221230801190, 9000017429201190, 9000001995301190, 9000177406501190, 9000166168101190, 9000179275301190, 9000052455801190, 9000062030201190, 9000078158201190, 9000127182201190, 9000217575801190, 9000166171001190, 9000002228601190, 9000166173901190, 9000062826701190, 9000215658701190, 9000229149901190, 9000002897101190, 9000228893901190, 9000022055201190, 9000029319201190, 9000166176801190, 9000059706401190, 9000002029601190, 9000025607201190, 9000041837601190, 9000208263201190, 9000233095201190, 9000167843701190, 9000227952501190, 9000211158901190, 9000001188119001, 9000185689801190, 9000117875401190, 9000041638601190, 9000022925001190, 9000046041801190, 9000033344201190, 9000030070301190, 9000002908701190, 9000166185501190, 9000170793501190, 9000019548701190, 9000020601201190, 9000093202801190, 9000083986801190, 9000060255601190, 9000077791601190, 9000143970501190, 9000024751301190, 9000060975301190, 9000002172101190, 9000052092101190, 9000139186201190, 9000139391001190, 9000106981401190, 9000166168601190, 9000161128301190, 9000030158701190, 9000042088301190, 9000040987501190, 9000201601901190, 9000211560301190, 9000095876801190, 9000029726401190, 9000166174401190, 9000179921601190, 9000186321601190, 9000024913601190, 9000132100801190, 9000105118401190, 9000092241601190, 9000176442901190, 9000059706901190, 9000041735701190, 9000124628501190, 9000166177301190, 9000021998101190, 9000145441301190, 9000133870101190, 9000118205801190, 9000021796201190, 9000112676201190, 9000166180201190, 9000086948201190, 9000059709801190, 9000166157501190, 9000117875901190, 9000157786301190, 9000023770301190, 9000119002301190, 9000211162301190, 9000041639101190, 9000211546301190, 9000118493201190, 9000170794001190, 9000166186001190, 9000072592401190, 9000001936401190, 9000211165201190, 9000166160401190, 9000024694801190, 9000170976101190, 9000166163301190, 9000030150501190, 9000030304101190, 9000168134501190, 9000059718501190, 9000186617701190, 9000166188901190, 9000031840101190, 9000205587301190, 4000001274119001, 9000017430201190, 9000119599801190, 9000002172601190, 9000013129401190, 9000220847801190, 9000072345101190, 9000060492301190, 9000017945101190, 9000026085901190, 9000032139801190, 6000001331119001, 9000166172001190, 9000211560801190, 9000101071201190, 9000158415201190, 9000166223201190, 9000118760801190, 9000217807201190, 9000180485301190, 9000020434101190, 9000023762101190, 9000034670601190, 9000166177801190, 9000119534601190, 9000033416201190, 9000002903901190, 9000117233501190, 9000166180701190, 9000023051101190, 9000041519201190, 9000184052401190, 9000115879601190, 9000179444401190, 9000211162801190, 9000133415601190, 9000221633201190, 9000211165701190, 9000026615301190, 9000166160901190, 9000062020101190, 9000118493701190, 9000171818501190, 9000029943301190, 9000211242501190, 9000119594501190, 9000002167301190, 9000166189401190, 9000166163801190, 9000062947501190, 9000032253101190, 9000207897601190, 9000184601601190, 9000176844801190, 9000214604801190, 9000070300501190, 9000001999701190, 9000119606101190, 9000060495701190, 1104119001, 9000119609001190, 9000211154601190, 9000030162601190, 9000166175401190, 9000119532201190, 9000061781501190, 9000118997501190, 9000166178301190, 9000118488401190, 9000120587601190, 9000004210001190, 9000127243601190, 9000116645201190, 9000002187601190, 9000004312401190, 9000211160401190, 9000036030801190, 9000045812901190, 9000204123301190, 9000166158501190, 9000118235301190, 9000039592101190, 9000119540901190, 9000213646501190, 3000001218119001, 9000120593401190, 9000028254201190, 9000060254201190, 9000119595001190, 9000029611001190, 9000212318201190, 9000211166201190, 9000001940301190, 9000001914701190, 9000187258701190, 9000021114701190, 9000060973901190, 9000023623501190, 9000194836301190, 9000001275119001, 9000103728801190, 9000025648801190, 9000018480801190, 9000042109601190, 9000028157601190, 9000108234401190, 9000166170101190, 9000119603701190, 9000204877301190, 9000024349001190, 9000047209801190, 9000024195401190, 9000062288201190, 9000034230601190, 9000131766601190, 9000055453001190, 9000026374301190, 9000175340701190, 9000166175901190, 9000151865501190, 6000001261119001, 8000001133119001, 9000024162401190, 9000166178801190, 9000118998001190, 9000016879601190, 9000138428401190, 9000040636401190, 9000211158001190, 9000053592901190, 9000041765701190, 9000118488901190, 9000022284101190, 9000211160901190, 9000027762501190, 9000010050201190, 9000202690201190, 9000212776601190, 9000183669401190, 9000001935001190, 9000128475801190, 9000048808601190, 9000019013101190, 9000174046701190, 9000211166701190, 9000060254701190, 9000166161901190, 9000002705901190, 9000002117101190, 4000001119119001, 9000097505601190, 9000021115201190, 9000120110401190, 9000034094401190, 9000166190401190, 9000119163201190, 9000118474901190, 9000118756501190, 9000078154901190, 9000138775701190, 9000004094101190, 9000039092201190, 9000118196201190, 9000166170601190, 9000060493801190, 9000023629801190, 9000059879401190, 9000111619901190, 9000160976701190, 9000024349501190, 9000062851901190, 9000166173501190, 9000034231101190, 9000029341501190, 9000152938301190, 9000105808701190, 9000002205501190, 9000119533201190, 9000155373201190, 9000118483601190, 9000215328401190, 9000211158501190, 9000052156901190, 9000208163301190, 9000023638501190, 9000002057701190, 9000004361701190, 1000001034119001, 9000211545401190, 9000023615801190, 9000029941901190, 9000211164301190, 9000020418701190, 9000048809101190, 5000001219119001, 1000001091119001, 9000067858401190, 9000112018401190, 9000029944801190, 9000061253601190, 9000187564001190, 9000041567201190, 9000046968801190, 9000043359201190, 9000166165301190, 9000077051701190, 9000002171701190, 9000028846901190, 9000042110601190, 9000118757001190, 9000023806601190, 9000084760201190, 9000051557001190, 9000026289801190, 9000150526601190, 9000183937501190, 9000025908701190, 9000062004701190, 9000085403101190, 9000028263901190, 9000062030301190, 9000023246301190, 9000002922801190, 9000002103601190, 9000106244401190, 9000179921201190, 9000227281201190, 9000229150001190, 9000077242501190, 9000041453701190, 9000118484101190, 9000041735301190, 9000006484101190, 9000002007001190, 9000062832601190, 9000172503001190, 9000055690201190, 9000059709401190, 9000034083801190, 9000150256601190, 9000211235801190, 9000211568601190, 9000117875501190, 9000001958701190, 9000116646701190, 9000211160601190, 1063119001, 9000001319119001, 7000001191119001, 9000211164801190, 9000030070401190, 9000082243201190, 9000166185601190, 9000039260801190, 9000029609601190, 9000230902401190, 9000117599701190, 9000060255701190, 9000166188501190, 9000034066901190, 9000166162901190, 9000112018901190, 9000002220501190, 9000051935701190, 9000166165801190, 9000025391401190, 9000023625001190, 9000034069801190, 9000211554601190, 9000139186301190, 9000062027901190, 9000004453501190, 9000066379901190, 9000028927101190, 9000002075601190, 9000022504401190, 9000189288401190, 9000211563301190, 9000002180901190, 9000095876901190, 9000029316901190, 9000186321701190, 9000126340901190, 9000041735801190, 9000171758201190, 9000223854201190, 9000042225101190, 9000059709901190, 9000023665101190, 9000075197901190, 9000025892301190, 9000146263501190, 9000017853901190, 9000211162401190, 9000181594401190, 9000166183201190, 9000118183201190, 9000156660001190, 9000062938401190, 9000023616801190, 9000022388001190, 9000211546401190, 9000132826401190, 9000183517301190, 9000029430901190, 9000028202101190, 5000001220119001, 9000030304201190, 9000118982601190, 9000125536201190, 9000029997001190, 9000167008201190, 9000002019101190, 9000030076701190, 9000025622301190, 9000020758301190, 9000043878001190, 9000119602801190, 9000030156401190, 9000166169201190, 9000060492401190, 9000210585201190, 9000003535301190, 9000190492101190, 9000130946501190, 9000095106501190, 9000211560901190, 9000217807301190, 9000229839301190, 6000001078119001, 9000062290201190, 9000166175001190, 9000119608601190, 9000025195801190, 9000211154201190, 9000179922201190, 9000026888301190, 9000166177901190, 9000104072301190, 9000046574701190, 9000127243201190, 9000166180801190, 9000022846401190, 9000124222401190, 9000022155201190, 9000028097301190, 9000034369301190, 9000060711701190, 9000167796501190, 9000042228501190, 9000166158101190, 9000211162901190, 9000211546901190, 9000002039401190, 9000170794601190, 9000118493801190, 9000028999101190, 9000026336701190, 9000035373501190, 9000098736401190, 9000119600401190, 9000218237201190, 9000211558501190, 9000166169701190, 9000028032101190, 9000207897701190, 9000107545701190, 9000170265701190, 1000001178119001, 6000001178119001, 9000042115001190, 9000025142201190, 9000060495801190, 9000001951501190, 9000119609101190, 9000025608801190, 9000002594401190, 9000042223201190, 9000166178401190, 9000184943201190, 9000117871201190, 9000043042401190, 9000211160501190, 9000127243701190, 9000118488501190, 9000127602101190, 9000206373301190, 9000001985801190, 9000166158601190, 9000164264201190, 9000211547401190, 9000029608201190, 9000212318301190, 9000091614301190, 9000060254301190, 9000029611101190, 9000142689201190, 9000029434801190, 9000055214001190, 9000166164401190, 9000187258801190, 9000123133701190, 9000042314501190, 9000052784901190, 9000172055301190, 9000023268101190, 9000139722501190, 2000001278119001, 9000166170201190, 9000092237401190, 9000033255001190, 9000176666201190, 9000060493401190, 9000062848601190, 9000017434201190, 9000109415001190, 9000031309401190, 9000211559001190, 9000007325101190, 9000119606701190, 9000180486401190, 9000062291201190, 9000118483201190, 9000211158101190, 9000061782101190, 9000030038101190, 9000062012501190, 9000117871701190, 9000118998101190, 9000002031701190, 9000082569301190, 9000211567701190, 6000001136119001, 9000081829801190, 9000037490601190, 9000061349801190, 9000029938601190, 9000166181801190, 9000118489001190, 9000211161001190, 9000211545001190, 9000211570601190, 9000105563901190, 9000172738301190, 9000202690301190, 9000165595901190, 9000100315901190, 1321119001, 9000187563601190, 9000166187601190, 9000002091601190, 9000123461201190, 9000060254801190, 9000119237201190, 9000187256401190, 9000047096401190, 9000001963601190, 9000211166801190, 9000166162001190, 9000058209701190, 9000184212901190, 9000001992101190, 9000211553701190, 9000071470501190, 9000002071801190, 9000166167801190, 9000042110201190, 9000143460601190, 9000093694201190, 9000118475001190, 9000002100301190, 9000119604301190, 9000073780301190, 9000221620301190, 9000033844301190, 9000211152801190, 9000004023201190, 9000166173601190, 9000173318901190, 9000155373301190, 9000141677301190, 9000118483701190, 9000166176501190, 9000062291701190, 9000146080501190, 9000023635701190, 9000150253301190, 9000034080501190, 8000001236119001, 9000062013001190, 9000022333001190, 9000042224201190, 9000034032201190, 9000211158601190, 9000052157001190, 9000201254301190, 9000002111901190, 9000116646301190, 9000119539101190, 9000001037119001, 1000001293119001, 9000166185201190, 9000211164401190, 9000116114501190, 9000123461701190, 9000166188101190, 9000060255301190, 9000117599301190, 9000021010501190, 9000166162501190, 9000221612101190, 9000020629401190, 9000063816601190, 9000118757101190, 9000179275501190, 9000030616301190, 9000180683501190, 9000001995501190, 9000062027501190, 9000026136301190, 5000001279119001, 9000176411201190, 9000161435201190, 9000034280001190, 9000218395201190, 9000001796501190, 9000034078101190, 9000101303701190, 9000166174101190, 9000031620501190, 9000080132501190, 9000041735401190, 9000035309801190, 9000047469801190, 9000141677801190, 9000166177001190, 9000060500201190, 9000023738601190, 9000119533801190, 9000030113001190, 9000018647101190, 9000211568701190, 9000062832701190, 9000038128701190, 9000183434301190, 9000001904701190, 9000211159101190, 9000211235901190, 9000121021501190, 9000166157201190, 9000041638801190, 9000116646801190, 9000039491301190, 9000045891301190, 9000062019301190, 9000230902501190, 9000117599801190, 9000214470201190, 9000154591801190, 9000166188601190, 9000060255801190, 9000230470201190, 9000075337101190, 9000211554701190, 9000117884301190, 9000005730701190, 9000080661901190, 9000007625101190, 9000166165901190, 9000026303001190, 9000186751201190, 9000004530401190, 9000029567201190, 9000023602401190, 9000219928801190, 9000190491701190, 9000024578101190, 9000089294901190, 9000002101301190, 1000001308119001, 2000001052119001, 1308119001, 9000062289801190, 9000132101001190, 9000029317001190, 9000023736201190, 9000039352201190, 9000002181001190, 9000119608201190, 9000026324701190, 9000166177501190, 9000005870301190, 9000041761501190, 9000023867101190, 9000047470301190, 9000001723101190, 5000001237119001, 9000032983601190, 9000041457201190, 9000025662001190, 9000222456401190, 9000036900401190, 9000166234501190, 9000101056901190, 9000002906501190, 9000211162501190, 9000020621701190, 3000001166119001, 9000122768601190, 9000072592601190, 9000143274201190, 9000024695001190, 9000053725401190, 9000211165401190, 9000028202201190, 9000118493401190, 9000211242201190, 9000219152601190, 9000060256301190, 9000071392301190, 9000177990701190, 9000117856301190, 9000049453101190, 9000017965101190, 9000211552301190, 9000116653101190, 9000166189101190, 9000032304001190, 9000020630401190, 9000166166401190, 9000183011201190, 9000060976001190, 9000160281301190, 9000166169301190, 9000106982101190, 9000004684501190, 9000010854101190, 7000001024119001, 9000176412201190, 9000060495401190, 9000029673001190, 9000118761001190, 9000083074601190, 9000166223401190, 9000031900201190, 9000003535401190, 9000166175101190, 9000178923901190, 9000211563901190, 9000166178001190, 9000084718801190, 9000094011601190, 9000055076901190, 9000127243301190, 9000166180901190, 3000001266119001, 9000211163001190, 9000118158201190, 9000026382201190, 9000166183801190, 9000174734201190, 9000219150201190, 9000211547001190, 9000025028301190, 9000197802701190, 9000170794701190, 9000013713101190, 9000118493901190, 9000173329101190, 9000214775501190, 9000187258401190, 9000025722401190, 9000031661601190, 9000134112801190, 9000051911201190, 9000032045601190, 9000087216501190, 9000166141301190, 9000062848201190, 9000209894601190, 9000031155401190, 9000002611401190, 9000197581001190, 9000222131401190, 9000003510301190, 9000003126301190, 9000031798301190, 9000138550301190, 9000125519901190, 9000025836401190, 9000025785201190, 9000041452401190, 9000119481201190, 9000112774001190, 9000117871301190, 9000104072901190, 9000030037701190, 9000211157701190, 9000202968601190, 9000001983001190, 9000119538201190, 9000025202201190, 9000116645401190, 9000127243801190, 9000118488601190, 9000166181401190, 9000001906201190, 9000041765401190, 9000029941101190, 9000025281901190, 9000166184301190, 9000039387501190, 9000029608301190, 9000045659501190, 1295119001, 9000046481601190, 9000027947201190, 9000060254401190, 9000100728001190, 9000031505601190, 9000137720001190, 9000076203201190, 9000039057601190, 9000212318401190, 9000097505301190, 9000166164501190, 9000187258901190, 9000117601301190, 9000031841301190, 9000166190101190, 9000045742101190, 9000185031701190, 9000192279401190, 9000171108201190, 9000166141801190, 9000118474601190, 9000029309801190, 9000166167401190, 9000119524201190, 9000029799101190, 9000119603901190, 9000028544701190, 9000060493501190, 9000222131901190, 9000152013501190, 1000001025119001, 9000002000401190, 9000166173201190, 9000193795601190, 9000077290001190, 9000042115601190, 9000151865701190, 9000118483301190, 9000166176101190, 9000017388901190, 9000002054501190, 9000002233701190, 9000062291301190, 8000001082119001, 9000003798201190, 9000173910201190, 9000044886201190, 9000002569401190, 9000117871801190, 9000062012601190, 9000211158201190, 9000022332601190, 9000031062201190, 9000023308301190, 9000166181901190, 9000203993101190, 9000211161101190, 9000001932301190, 9000005797901190, 9000041765901190, 9000118489101190, 8000001139119001, 9000211548001190, 9000091458401190, 9000165596001190, 9000029941601190, 9000030069601190, 9000179164001190, 9000018447201190, 9000043151201190, 9000135132001190, 9000211166901190, 9000204536501190, 9000166162101190, 9000034398901190, 9000166187701190, 9000061253301190, 9000048043701190, 9000060254901190, 9000212318901190, 9000166190601190, 9000119598601190, 9000211553801190, 9000054475101190, 9000118475101190, 9000202010801190, 9000033844401190, 9000036967601190, 9000002228401190, 9000204801201190, 9000193796101190, 9000041476101190, 9000156087301190, 9000020714501190, 9000062826501190, 9000166173701190, 6000001310119001, 9000038842201190, 9000160954201190, 9000156141401190, 9000025607001190, 9000176442201190, 9000118483801190, 9000033008301190, 9000004669101190, 9000211235501190, 9000211158701190, 9000125961901190, 9000138429101190, 9000042227201190, 9000117875201190, 9000132134401190, 9000023923201190, 9000104076801190, 9000185689601190, 9000132979201190, 9000046835201190, 9000211545601190, 9000166182401190, 9000030172501190, 9000182876501190, 9000166185301190, 9000170793301190, 9000062021801190, 9000074924201190, 9000046969001190, 9000166188201190, 9000060255401190, 9000117599401190, 9000149599401190, 9000181292201190, 9000171513001190, 9000053010601190, 9000212322301190, 9000006114301190, 9000062843901190, 9000169263101190, 9000032840701190, 1000001097119001, 9000029310801190, 9000001995601190, 9000205157201190, 9000166168401190, 9000042110801190, 9000152600401190, 9000037707601190, 9000041573201190, 9000167192401190, 9000233541201190, 9000008219301190, 9000072347301190, 9000040116901190, 9000028264101190, 9000027163301190, 9000002049701190, 9000002052601190, 9000101303801190, 9000023479801190, 9000094622201190, 9000049540601190, 9000166174201190, 9000041399801190, 9000227281401190, 9000002846201190, 9000042221901190, 9000041735501190, 9000211156301190, 9000041761101190, 9000166177101190, 9000060500301190, 9000059706701190, 9000024839501190, 9000034081101190, 9000016983201190, 9000160983201190, 9000188557301190, 9000166157301190, 9000116646901190, 9000117875701190, 9000064678901190, 9000016909301190, 9000211546101190, 9000079450101190, 7000001268119001, 9000166185801190, 9000166160201190, 9000172944201190, 9000230902601190, 9000128300701190, 9000060255901190, 9000166163101190, 9000023647901190, 9000007625201190, 9000060975601190, 9000166166001190, 9000020604401190, 9000022908401190, 9000182322501190, 9000117528901190, 9000186751301190, 9000213759301190, 9000047823001190, 9000042114201190, 9000034306201190, 9000060981401190, 9000020968601190, 9000179970201190, 9000211563501190, 9000192696301190, 9000088504301190, 9000166174701190, 9000030417901190, 9000145902401190, 9000031393601190, 9000002058901190, 9000029937301190, 9000166180501190, 9000075198101190, 9000032138901190, 9000116388501190, 9000117233301190, 9000042228201190, 9000001933801190, 9000166157801190, 9000116647401190, 9000211162601190, 9000171818301190, 9000118493501190, 9000030813501190, 9000211165501190, 9000170794301190, 9000166160701190, 9000172637501190, 9000211242301190, 4000001169119001, 9000025619601190, 9000060256401190, 9000134112401190, 9000166163601190, 9000118982801190, 9000207894501190, 9000119600101190, 9000122262501190, 9000143459301190, 9000166166501190, 9000165836601190, 9000047795001190, 9000029567801190, 9000214604601190, 9000149529401190, 9000161385101190, 9000023733901190, 9000060495501190, 9000023017101190, 9000001922701190, 9000001283119001, 9000211564001190, 9000070098401190, 9000062293301190, 9000166178101190, 9000020770101190, 6000001212119001, 9000105278601190, 9000211544201190, 9000118488201190, 9000166181001190, 9000124939401190, 9000115211401190, 9000017342601190, 9000211160201190, 9000002011101190, 9000060711901190, 9000211163101190, 9000106510301190, 9000212315101190, 9000046247901190, 9000001141119001, 9000118494001190, 9000153028401190, 9000166161201190, 9000047044401190, 9000159889201190, 9000214775601190, 9000060254001190, 9000004548401190, 9000211242801190, 9000211166001190, 9000134112901190, 9000187258501190, 9000158205401190, 9000062947801190, 9000042109401190, 9000025623001190, 9000223229401190, 9000107545901190, 9000031155501190, 9000023629101190, 9000188006701190, 9000209894701190, 9000222131501190, 9000184601901190, 9000117888301190, 9000211570201190, 9000042115201190, 9000207542401190, 9000111619201190, 9000034079701190, 9000050463701190, 9000226002901190, 9000119609301190, 9000211567401190, 9000002057001190, 9000166178601190, 9000216431401190, 9000184943401190, 9000211157801190, 9000001931901190, 9000166181501190, 9000048549501190, 9000181592701190, 9000211160701190, 9000118488701190, 9000211544701190, 4000001113119001, 1113119001, 9000164264401190, 9000048808401190, 9000029941201190, 9000135131601190, 9000034065701190, 9000029944101190, 9000137720101190, 9000060254501190, 9000211166501190, 9000120593701190, 9000212318501190, 9000031841401190, 9000187259001190, 9000166190201190, 9000042030201190, 9000185031801190, 9000211553401190, 9000158743501190, 9000054474701190, 9000118474701190, 9000166167501190, 9000209895201190, 9000060493601190, 9000207898401190, 9000205159201190, 9000048797301190, 9000034230901190, 9000038454901190, 4000001284119001, 9000118483401190, 9000166176201190, 9000049286601190, 9000001952201190, 9000180486601190, 9000207036701190, 9000182307101190, 9000023638301190, 9000027452701190, 9000059222301190, 9000211567901190, 9000025635101190, 9000119538801190, 9000130009201190, 9000081830001190, 9000020594801190, 9000018063301190, 9000034959301190, 9000211548101190, 9000213365701190, 9000219458501190, 9000025103301190, 9000163036101190, 9000118236101190, 9000033807301190, 6000001142119001, 9000047096601190, 9000001938201190, 9000042104601190, 9000060255001190, 9000087749401190, 9000166162201190, 9000187256601190, 9000022981401190, 9000211167001190, 9000232491801190, 9000034065201190, 9000062843501190, 9000002145901190, 9000071470701190, 9000166190701190, 9000149755501190, 7000001327119001, 9000118756801190, 9000119601601190, 9000002148801190, 9000025626901190, 9000166173801190, 9000177642601190, 9000047978601190, 9000211562601190, 9000215658601190, 9000166176701190, 9000030112701190, 9000118483901190, 9000156141501190, 9000002003901190, 9000176442301190, 8000001185119001, 9000116387601190, 9000027453201190, 9000020003601190, 9000167843601190, 9000062294801190, 9000211158801190, 9000211235601190, 9000214464101190, 9000080780901190, 9000041638501190, 9000041766501190, 9000185689701190, 9000116646501190, 9000017344101190, 9000117852601190, 9000166185401190, 9000211164601190, 9000225398201190, 9000230902201190, 9000059538701190, 9000074924301190, 9000062021901190, 9000166162701190, 9000060255501190, 9000062841101190, 9000029996301190, 9000030562401190, 9000143970401190, 9000025647201190, 9000119599201190, 9000030152801190, 9000026773601190, 9000060975201190, 9000150191201190, 9000010927201190, 9000002120801190, 9000180683701190, 9000031435701190, 9000074981301190, 9000219928501190, 9000106981301190, 9000114942901190, 9000161128201190, 9000024936201190, 9000161435401190, 9000110389001190, 9000118760201190, 9000223413001190, 9000027857501190, 9000186321501190, 9000023479901190, 9000030417501190, 9000166174301190, 9000015083101190, 9000005611101190, 9000211563101190, 6000001157119001, 9000030497201190, 9000023329201190, 9000059706801190, 9000054689201190, 9000041735601190, 9000037821701190, 9000059709701190, 9000172503301190, 9000002570501190, 9000004644101190, 9000117875801190, 9000015680601190, 9000132826201190, 9000119002201190, 9000166234201190, 9000101056601190, 9000211162201190, 9000181594201190, 9000211165101190, 9000166160301190, 9000023952301190, 9000061942701190, 9000166185901190, 9000017373101190, 9000116647201190, 7000001143119001, 9000060256001190, 9000006675201190, 9000023622401190, 9000166188801190, 9000002044501190, 9000054729301190, 9000002198101190, 9000060975701190, 9000158588501190, 9000109439401190, 9000101605801190, 9000066380201190, 9000166169001190, 9000225689001190, 9000073625001190, 9000220364201190, 9000003125501190, 9000025653501190, 9000166171901190, 9000169679101190, 9000012341501190, 9000060981501190, 9000105118801190, 9000002642001190, 9000088504401190, 9000002181201190, 9000119534501190, 9000054689701190, 9000166177701190, 9000110395301190, 9000145902501190, 9000001672101190, 9000024689401190, 9000166180601190, 9000029937401190, 9000112676601190, 9000002110201190, 9000117233401190, 9000221860601190, 9000212570701190, 9000023873101190, 9000061965901190, 9000101057101190, 9000184052301190, 9000017857101190, 9000211162701190, 9000069264801190, 9000183517601190, 9000164880801190, 9000029354401190, 9000166160801190, 9000211165601190, 9000125536501190, 9000070675701190, 4000001172119001, 9000062947401190, 9000087216201190, 9000030077001190, 9000020579401190, 9000001942601190, 9000027135901190, 9000205849501190, 9000060492701190, 9000166169501190, 9000160281501190, 9000032307101190, 9000207897501190, 7000001101119001, 7000001229119001, 9000166172401190, 9000039631601190, 9000229839601190, 9000090066501190, 9000166175301190, 9000181407301190, 9000104072601190, 9000062011801190, 9000166178201190, 9000119535001190, 9000211157401190, 9000211160301190, 9000118488301190, 9000042225901190, 9000166181101190, 9000154689601190, 9000211547201190, 9000166184001190, 9000034958401190, 9000002036801190, 9000166158401190, 9000060254101190, 9000120593301190, 9000116113301190, 9000001911701190, 9000214775701190, 9000211166101190, 9000166164201190, 9000112020201190, 9000139181801190, 9000028999401190, 9000029946601190, 9000187258601190, 9000051911401190, 9000166189801190, 9000211553001190, 9000164884201190, 9000123133501190, 9000140464701190, 9000028083601190, 9000060493201190, 9000033254801190, 9000215117201190, 9000166170001190, 9000133094801190, 9000209894801190, 9000232141201190, 9000166172901190, 9000196048101190, 9000025654501190, 9000192694501190, 9000021456101190, 9000070096101190, 9000111619301190, 9000136195301190, 9000098025701190, 9000101305401190, 9000157369401190, 9000062521401190, 9000166175801190, 9000119532601190, 9000002028601190, 9000193337401190, 9000211567501190, 9000186812301190, 9000021052301190, 9000041762701190, 9000138428301190, 9000211157901190, 2000001187119001, 9000025637601190, 9000188632801190, 9000211160801190, 9000211570401190, 9000119538401190, 9000166181601190, 9000118488801190, 9000116645601190, 9000029608501190, 9000025333301190, 9000017986101190, 9000211547701190, 9000216206901190, 3000001244119001, 9000212318601190, 9000028254601190, 9000031531401190, 9000060254601190, 9000037905801190, 9000005624201190, 9000033195401190]